In [34]:
class Stock:
    def __init__(self):
        self.empresa = ''
        self.codcvm = 0
        self.data_divulg = ''
        self.precoinicio = 0
        self.precofim = 0
        self.ultimainfo = ''
        self.dados_financeiros = []
        self.indicadorfinanceiro = 0

    def inputvalores_primarios(self, id, cod_cvm, data_divulgacao, ultimo_trimestre, dados_itr):
        self.empresa = id
        self.codcvm = cod_cvm
        self.data_divulg = data_divulgacao
        self.ultimainfo = ultimo_trimestre
        self.dados_financeiros = dados_itr

    def inputvalores_precos(self, preco_entrada, preco_saida):
        self.precoinicio = preco_entrada
        self.precofim = preco_saida

    def show_info(self):
        print(f'Nome empresa: {self.empresa}')
        print(f'Cod cvm: {self.codcvm}')
        print(f'Data divulgacao: {self.data_divulg}')
        print(f'Preco Inicio: {self.precoinicio}')
        print(f'Preco Fim: {self.precofim}')
        print(f'Ultimo Trimestre: {self.ultimainfo}')        
        print(f'Dados do ITRo: {self.dados_financeiros}')
        print(f'Dados do ITRo: {self.indicadorfinanceiro}')        

import pandas as pd
import mysql.connector
# import datetime as datetime
import warnings
from datetime import datetime, date, timedelta
warnings.filterwarnings('ignore')

db = mysql.connector.connect(
    user='cartor',
    password='Cartor1212',
    host='ddns.cartor.com.br',
    port='53306', # 43306
    database='console' # os.getenv('MYSQL_DATABASE'),
    )

def encontrar_close(df, stock_name, fdata_divulgacao):
    try:
        filtro = (df['Stock'] == stock_name) & (df['Date'] == fdata_divulgacao)
        close_value = df.loc[filtro, 'Close'].values[0]
        return close_value
    except IndexError:
        try:
            filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=1)))
            close_value = df.loc[filtro, 'Close'].values[0]
            return close_value            
        except IndexError:        
            try:
                filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=2)))
                close_value = df.loc[filtro, 'Close'].values[0]
                return close_value
            except IndexError:
                try:
                    filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=2)))
                    close_value = df.loc[filtro, 'Close'].values[0]
                    return close_value
                except IndexError:                                                                
                    return None 
    
def converterstr_data(data_hora_string):
    data_hora = datetime.strptime(data_hora_string, "%d/%m/%Y %H:%M:%S")
    data = data_hora.date()
    return data    

def extrair_hora(data_hora_str):
    data_hora = datetime.strptime(data_hora_str, "%d/%m/%Y %H:%M:%S")
    horas = data_hora.hour
    if horas > 17:
        aumentar_dias = 1
    else:
        aumentar_dias = 0
    return aumentar_dias

msql = pd.read_sql_query("SELECT * from b3", db)
msql = msql.reset_index(drop=True)         

In [35]:
from pymongo import MongoClient
import re

# Acessando o banco MongoDB Compass
cliente = MongoClient(
    "ddns.cartor.com.br",
    port=57017,
    username='cartor',
    password='Cartor1212',
    authSource='admin',
    authMechanism='DEFAULT'
)

db = cliente.CVM
collection = 'Empresas_fin6'

def finddoc_byequity(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byid(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byname(db, collection_name, containing_word, trimestre=False):
    count = 0
    while count != 1:

        if trimestre:
            collection = db[collection_name]
            regex = re.compile(containing_word+trimestre, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        else:
            collection = db[collection_name]
            regex = re.compile(containing_word, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})

        if count > 1:
            print(type(documents))
            print('erro aconteceu - checar base')
            for doc in documents:
                print(doc['_id'])
            break
            # containing_word = input("Qual das empresas seria a solicitada: ")
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def balance_type(f_empresa, f_trimestre):
    tipo_balanco = 'ERROR'   
    for key in f_empresa[f_trimestre[0]]:
        key2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', key)       
        try:
             pvalor = f_empresa[f_trimestre[0]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
             if pvalor != 0: 
                 tipo_balanco = key
                 break
        except KeyError:
            print('\nCia nao possui DF CONSOLIDADO')
    novalista_trimestres = []
    if tipo_balanco == 'DFConsolidado':
        for trimestre in f_trimestre:
            if len(f_empresa[trimestre].get(tipo_balanco, {})) != 0:
                    novalista_trimestres.append(trimestre) 
        f_trimestre = novalista_trimestres
    print(f'\nUtilizando balanço: {tipo_balanco}')        
    return tipo_balanco, f_trimestre

def transform_FDC_tri(f_dicionario, f_conta, f_quarter, f_qtdtrimestres, f_trimestres, f_tipobalanco, f_tipobalanco2):
    f_valor = 0
    f_total_tri = f_quarter+1
    if f_total_tri > f_qtdtrimestres:
        try:
            f_valor = f_dicionario[f_trimestres[f_quarter]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'][f_conta]['vl_conta']
        except KeyError:
            f_valor = 0
        dt = datetime.strptime(f_trimestres[f_quarter], '%Y-%m-%d')
        if dt.month == 12: f_valor /= 4
        elif dt.month == 9: f_valor /= 3
        elif dt.month == 6: f_valor /= 2
    else:
        # print(f_trimestres[f_quarter], ",", f_conta)
        try:
            f_valor = f_dicionario[f_trimestres[f_quarter]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'][f_conta]['vl_conta']
        except KeyError:
            try:
                f_valor = f_dicionario[f_trimestres[f_quarter]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Direto)'][f_conta]['vl_conta']
            except KeyError:
                f_valor = 0
        dt = datetime.strptime(f_trimestres[f_quarter], '%Y-%m-%d')
        if dt.month != 3:
            try:
                f_valor -= f_dicionario[f_trimestres[f_total_tri]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'][f_conta]['vl_conta']
            except KeyError:
                f_valor = 0
    return f_valor

def calculo_variacao_trimacum(dados_financerios, trimestres_inicial, trimestres_final):
    index_inicial = dados_financerios.index(trimestres_inicial)
    index_final = dados_financerios.index(trimestres_final)
    return index_inicial, index_final

def ratio_indicadores(index_final, index_inicial, dados_financeiros):    
    soma1 = 0
    for i in range(index_final, index_final+4):
        soma1 += dados_financeiros[i]
    soma2 = 0    
    for j in range(index_inicial, index_inicial+4):
        soma2 += dados_financeiros[j]    
    return round(soma1/soma2, 4)


In [36]:
def upload_infofin(data_fin):
    trimestres_upload = []
    for chave, valor in data_fin.items():
        if isinstance(chave, str) and chave.startswith('2'):
            trimestres_upload.append(chave)
    trimestres_upload = sorted(trimestres_upload, reverse=True)     
    tipo_balanco, trimestres_upload = balance_type(df1, trimestres_upload)
    if tipo_balanco == 'ERROR': exit()
    tipo_balanco2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', tipo_balanco)
    return tipo_balanco, tipo_balanco2, trimestres_upload

In [37]:
ROMI3 = Stock()
WEGE3 = Stock()
ASAI3 = Stock()

In [39]:
stock_list = ['ROMI3', 'WEGE3', 'ASAI3']
stock_info = [ ROMI3, WEGE3, ASAI3 ]

collection = 'divulgacaoinfo_resultados'

for i, stock in enumerate(stock_list):

    # log no banco 1
    document = finddoc_byequity(db, 'Empresas_fin6', stock)
    df1 = document.next() # dicionario referente a stock pick
    tipo1, tipo2, trimestres_lista = upload_infofin(df1)
    fdc = []
    qtd_trimestres = len(trimestres_lista)
    print(f'Qtd de trimestres: {len(trimestres_lista)}')
    for j in range(len(trimestres_lista)-1):
        fdc.append(transform_FDC_tri(df1,'6.01.01', j, qtd_trimestres, trimestres_lista, tipo1, tipo2))

    # log no banco 2
    id_empresa = df1["_id"]    
    document = finddoc_byname(db, 'divulgacaoinfo_resultados', id_empresa, '_30/06/2023')
    df2 = document.next() # dicionario referente a stock pick
    id_divulgacao = df2["nome_empresa"]    

    # input no objeto
    Stock.inputvalores_primarios(stock_info[i], id_empresa, df2["cod_cvm"], df2["hora_envio"], trimestres_lista[0], fdc)
    shift_dias = extrair_hora(stock_info[i].data_divulg)    
    dia_posdivulgacao = converterstr_data(stock_info[i].data_divulg) + timedelta(shift_dias)
    preco_inicio = encontrar_close(msql, stock, dia_posdivulgacao)
    preco_fim = encontrar_close(msql, stock, converterstr_data('10/10/2023 00:00:00'))
    Stock.inputvalores_precos(stock_info[i], preco_inicio, preco_fim)

    # calculando os indicadores
    index1, index2 = calculo_variacao_trimacum(trimestres_lista, '2023-06-30', '2022-06-30')
    stock_info[i].indicadorfinanceiro = ratio_indicadores(index1, index2, stock_info[i].dados_financeiros)

    # show info empresas
    Stock.show_info(stock_info[i])
    print(" ")


Utilizando balanço: DFConsolidado
Qtd de trimestres: 23
Nome empresa: ROMI S.A.
Cod cvm: 7510
Data divulgacao: 18/07/2023 18:08:09
Preco Inicio: 14.11
Preco Fim: 11.96
Ultimo Trimestre: 2023-09-30
Dados do ITRo: [38386.0, 42614.0, 53335.0, 113759.0, 73060.0, 85765.0, 32439.0, 67482.0, 74243.0, 50128.0, 38502.0, 73804.0, 7282.0, 30780.0, 20833.0, 28004.0, 33504.0, 4385.0, 6733.0, 32126.0, 24988.0, 11726.0]
Dados do ITRo: 1.0879
 

Utilizando balanço: DFConsolidado
Qtd de trimestres: 23
Nome empresa: WEG S.A.
Cod cvm: 5410
Data divulgacao: 19/07/2023 07:03:26
Preco Inicio: 37.91
Preco Fim: 35.3
Ultimo Trimestre: 2023-09-30
Dados do ITRo: [1992628.0, 2162381.0, 2031939.0, 2221437.0, 1837567.0, 1409530.0, 1423174.0, 1234835.0, 1275623.0, 1176632.0, 1212713.0, 1137501.0, 1074490.0, 839340.0, 748825.0, 788884.0, 665578.0, 595432.0, 445763.0, 340968.0, 618443.0, 566678.0]
Dados do ITRo: 1.5447
 

Cia nao possui DF CONSOLIDADO

Utilizando balanço: DFIndividual
Qtd de trimestres: 16
Nome empre

In [21]:
print(trimestres_lista[index1])
print(trimestres_lista[index2])

2023-06-30
2022-06-30


In [ ]:
# PROXIMOS PASSOS:
# > inserir 10 empresas
# > fazer o rank com os dados dos indicadores e separar em quartil
# > verificar o retorno e pegar a média comparando com o ibovespa cada um deles. 